# Lab 1

Group Members:

- Parker

- Suma 

- Chris

- Oliver

## Introduction

**Why is it important to find this kind of bias in machine learning models?**

TODO: Answer this

**Why will the type of investigation I am performing be relevant to other researchers or practitioners?**

TODO: Answer this

For example, we are trying to answer if a romantic comedy is ranked as more positive or if a horror movie is ranked as more negative. 

To do this, we utilize the dataset of IMDB reviews that are pre-labeled as positive or negative and contain movie genres.
We will analyze the bias using the word embeddings of GloVe & ConceptNet. Each of these embeddings will be applied to the movie reviews to determine the overall sentiment of the review. 

We believe that a "lesser" embedding will perform more poorly in the face of a conflicting sentiment lexicon. By this, we expect that "horror" movies may have more "negative" ratings because of the "negative" words used in the reviews to describe the content of the movie, versus the overall context of the review as positive or negative. If the embedding is more narrow-focused, like Glove, it may produce results that bias towards romantic comedies as more positive, since the overall content of the review should have more "positive" words based on the content of the movie. However, we expect an embedding that has a wider knowledge graph focus, like ConceptNet, to remove this bias and focus solely on the review content.

## Investigation
As part of your assignment, you will choose a methodology that involves comparing two (or more) techniques to one another. Discuss how you will measure a difference between the two techniques. That is, if you are measuring the difference statistically, what test will you use and why is it appropriate? Are there any limitations to performing this test that you should be aware of?
Source: https://ieee-dataport.org/open-access/imdb-movie-reviews-dataset
Dataset has 1 million reviews from 1150 movies spread across 17 genres; there's also other meta data such as the IMDb rating and movie rating. 
Reference Paper: https://ieeexplore-ieee-org.proxy.libraries.smu.edu/document/9276893 

### Embeddings

We will be making our bias comparisons using the GLoVe and ConceptNet Numberbatch word embeddings. As iterated before, we believe that because of Numberbatch's broader generalization, we can prove that there exists a bias using GLoVe in which a romantic comedy movie, for example, may be rated as positive based on the inherent content of the review even though it received a negative numeric rating.

### Comparison metric

F-score

TODO: Explain why

### Hypothesis

TODO: Write hypothesis

## Training / Modelling

### Loading Embeddings and Lexicons

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import seaborn
import re

from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

/var/folders/kz/g4z3ghc90wz606t2zpb4l6bm0000gn/T/ipykernel_66175/2373303691.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
#this code is taken from LectureNotesMaster/01 ConceptNet.ipynb notebook shared by Prof E Larson/Robyn Speer

def load_embeddings(filename):
    """
    Load a DataFrame from the generalized text format used by word2vec, GloVe,
    fastText, and ConceptNet Numberbatch. The main point where they differ is
    whether there is an initial line with the dimensions of the matrix.
    """
    labels = []
    rows = []
    with open(filename, encoding='utf-8') as infile:
        for i, line in enumerate(infile):
            items = line.rstrip().split(' ')
            if len(items) == 2:
                # This is a header row giving the shape of the matrix
                continue
            labels.append(items[0])
            values = np.array([float(x) for x in items[1:]], 'f')
            rows.append(values)
    
    arr = np.vstack(rows)
    return pd.DataFrame(arr, index=labels, dtype='f')

embeddings = load_embeddings('data/glove.840B.300d.txt')
embeddings.shape

(2196018, 300)

In [4]:
#Loading the lexicon
def load_lexicon(filename):
    """
    Load a file from Bing Liu's sentiment lexicon
    (https://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html), containing
    English words in Latin-1 encoding.
    
    One file contains a list of positive words, and the other contains
    a list of negative words. The files contain comment lines starting
    with ';' and blank lines, which should be skipped.
    """
    lexicon = []
    with open(filename, encoding='latin-1') as infile:
        for line in infile:
            line = line.rstrip()
            if line and not line.startswith(';'):
                lexicon.append(line)
    return lexicon

pos_words = load_lexicon('data/positive-words.txt')
neg_words = load_lexicon('data/negative-words.txt')

print(len(pos_words), len(neg_words))

2006 4783


### Preparing Sentiment Classifier

In [5]:
#train the model
pos_words_common = list(set(pos_words) & set(embeddings.index)) 
neg_words_common = list(set(neg_words) & set(embeddings.index)) 

pos_vectors = embeddings.loc[pos_words_common]
neg_vectors = embeddings.loc[neg_words_common]
print(pos_vectors.shape,neg_vectors.shape)

(1974, 300) (4642, 300)


In [6]:
#train the inputs and outputs
vectors = pd.concat([pos_vectors, neg_vectors])
targets = np.array([1 for entry in pos_vectors.index] + [-1 for entry in neg_vectors.index])
labels = list(pos_vectors.index) + list(neg_vectors.index)

In [7]:
#prep for train test split
train_vectors, test_vectors, train_targets, test_targets, train_labels, test_labels = \
    train_test_split(vectors, targets, labels, test_size=0.1, random_state=0)

In [9]:
# create a linear classifier 
model = SGDClassifier(loss='log_loss', random_state=0, max_iter=100)
model.fit(train_vectors, train_targets)
accuracy_score(model.predict(test_vectors), test_targets)



0.9546827794561934

### Sentiment Analysis Functions

In [10]:
def vecs_to_sentiment(vecs):
    # predict_log_proba gives the log probability for each class
    predictions = model.predict_log_proba(vecs)

    # To see an overall positive vs. negative classification in one number,
    # we take the log probability of positive sentiment minus the log
    # probability of negative sentiment.
    # this is a logarithm of the max margin for the classifier, 
    # similar to odds ratio (but not exact) log(p_1/p_0) = log(p_1)-log(p_0)
    return predictions[:, 1] - predictions[:, 0]


def words_to_sentiment(words):
    # Filter the list of words to only include those present in the embeddings
    filtered_words = [word for word in words if word in embeddings.index]
    
    # If no words are found in the embeddings, return an empty DataFrame or some default sentiment value
    if not filtered_words:
        return pd.DataFrame({'sentiment': []})
    
    vecs = embeddings.loc[filtered_words].dropna()
    log_odds = vecs_to_sentiment(vecs)
    return pd.DataFrame({'sentiment': log_odds}, index=vecs.index)



# Show 20 examples from the test set
words_to_sentiment(test_labels).iloc[:20]

,sentiment
miserably,-8.467720
remission,5.897825
awfully,-6.856798
scared,-12.798662
sensationally,6.440469
momentous,4.026076
pride,3.956458
subpoena,-6.236719
anti-white,-6.282487
severity,-11.472931


In [11]:
#tokenize using regular expressions

TOKEN_RE = re.compile(r"\w.*?\b")
# The regex above finds tokens that start with a word-like character (\w), and continues
# matching characters (.+?) until the next word break (\b). It's a relatively simple
# expression that manages to extract something very much like words from text.


def text_to_sentiment(text):
    # Tokenize the input phrase
    tokens = [token.casefold() for token in TOKEN_RE.findall(text)]
    # Filter tokens to include only those that are in the embeddings' index
    tokens = [token for token in tokens if token in embeddings.index]
    
    # If no tokens are found in the embeddings, return a default sentiment value, e.g., 0
    if not tokens:
        return 0  # This is a neutral sentiment score
    
    sentiments = words_to_sentiment(tokens)
    if sentiments.empty:
        return 0  # Again, returning a neutral sentiment score if no sentiments were computed
    return sentiments['sentiment'].mean()  # Return the mean sentiment for the remaining tokens


### Loading Movie Data

In [12]:
#Load the movie list for Comedy
df_cat1_movies = pd.read_csv('data/movie_dataset/1_movies_per_genre/Comedy.csv', sep = ',')
df_cat1_movies.head()

,name,year,movie_rated,run_length,genres,release_date,rating,num_raters,num_reviews,review_url
0,Guardians of the Galaxy,2014,PG-13,2h 1min,Comedy;,1 August 2014 (USA),8.0,1017578,1390,https://www.imdb.com/title/tt2015381/reviews/_...
1,Back to the Future,1985,PG,1h 56min,Comedy;,3 July 1985 (USA),8.5,1015437,1178,https://www.imdb.com/title/tt0088763/reviews/_...
2,Finding Nemo,2003,G,1h 40min,Comedy;,30 May 2003 (USA),8.1,922237,968,https://www.imdb.com/title/tt0266543/reviews/_...
3,Up,2009,PG,1h 36min,Comedy;,29 May 2009 (USA),8.2,905205,891,https://www.imdb.com/title/tt1049413/reviews/_...
4,The Truman Show,1998,PG,1h 43min,Comedy;,5 June 1998 (USA),8.1,901368,1152,https://www.imdb.com/title/tt0120382/reviews/_...


In [13]:
df_cat2_movies = pd.read_csv('data/movie_dataset/1_movies_per_genre/Horror.csv', sep = ',')
df_cat2_movies.head()

,name,year,movie_rated,run_length,genres,release_date,rating,num_raters,num_reviews,review_url
0,The Shining,1980,R,2h 26min,Drama; Horror;,13 June 1980 (USA),8.4,860290,1805,https://www.imdb.com/title/tt0081505/reviews/_...
1,Alien,1979,R,1h 57min,Horror; Sci-Fi;,22 June 1979 (USA),8.4,762793,1423,https://www.imdb.com/title/tt0078748/reviews/_...
2,World War Z,2013,PG-13,1h 56min,Action; Adventure; Horror;,21 June 2013 (USA),7.0,586801,1125,https://www.imdb.com/title/tt0816711/reviews/_...
3,Psycho,1960,R,1h 49min,Horror; Mystery; Thriller;,8 September 1960 (USA),8.5,580951,1263,https://www.imdb.com/title/tt0054215/reviews/_...
4,Shaun of the Dead,2004,R,1h 39min,Comedy; Horror;,24 September 2004 (USA),7.9,499244,971,https://www.imdb.com/title/tt0365748/reviews/_...


In [14]:
#I feel name, year and genres cols in the csv files are sufficient to retain, drop the rest, add cols back in as needed
df_cat1_movies_lim=df_cat1_movies[['name', 'year',  'genres']]
df_cat2_movies_lim=df_cat2_movies[['name', 'year', 'genres']]

Interesting  problem - there is movie titled Evil Dead thats a Comedy;Horror mix. May be we need load Cat1 to some other genre?

For the movies in the dataframes we have loaded, bring in the reviews

In [15]:
df_concat_movies = pd.concat([df_cat1_movies_lim, df_cat2_movies_lim])
df_concat_movies['file_name'] = df_concat_movies['name'] + ' ' + df_concat_movies['year'].astype(str) + '.csv'

df_concat_movies.head()

,name,year,genres,file_name
0,Guardians of the Galaxy,2014,Comedy;,Guardians of the Galaxy 2014.csv
1,Back to the Future,1985,Comedy;,Back to the Future 1985.csv
2,Finding Nemo,2003,Comedy;,Finding Nemo 2003.csv
3,Up,2009,Comedy;,Up 2009.csv
4,The Truman Show,1998,Comedy;,The Truman Show 1998.csv


### Combining Movie Reviews

In [16]:
#https://www.geeksforgeeks.org/how-to-iterate-over-files-in-directory-using-python/
# and https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe
import os
directory = 'data/movie_dataset/2_reviews_per_movie_raw'

dfs = list()

# iterate over files in
# that directory
for filename in os.listdir(directory):	
	print('looking for :', filename) 
	if df_concat_movies['file_name'].eq(filename).any():
		print("Found:", df_concat_movies['file_name'])
		data = pd.read_csv(os.path.join(directory, filename), header=None)
		data['file_name'] = filename
#		data['genre'] = df_concat_movies['genres']        
		dfs.append(data)
df = pd.concat(dfs, ignore_index=True)

looking for : The Rock 1996.csv
looking for : The Double Life of Véronique 1991.csv
looking for : Captain America_ The Winter Soldier 2014.csv
looking for : Run, Fat Boy, Run 2007.csv
looking for : The Devil's Advocate 1997.csv
looking for : Ford v Ferrari 2019.csv
looking for : The Blair Witch Project 1999.csv
Found: 0     Guardians of the Galaxy 2014.csv
1          Back to the Future 1985.csv
2                Finding Nemo 2003.csv
3                          Up 2009.csv
4             The Truman Show 1998.csv
                    ...               
95                  From Hell 2001.csv
96              The Orphanage 2007.csv
97             Train to Busan 2016.csv
98               Evil Dead II 1987.csv
99       Underworld Awakening 2012.csv
Name: file_name, Length: 200, dtype: object
looking for : The King's Speech 2010.csv
looking for : Despicable Me 2 2013.csv
Found: 0     Guardians of the Galaxy 2014.csv
1          Back to the Future 1985.csv
2                Finding Nemo 2003.csv
3  

In [17]:
df = pd.DataFrame(df)
df = df.rename(columns={"filename": "file_name"})
df_all = df_concat_movies.merge(df)
df_all.columns = df_all.iloc[0]
df_all = df_all[1:]
df_all = df_all.reset_index(drop=True)
# Fixing a weird result from the merge - this seems to work ok
df_all = df_all.rename(columns={"Guardians of the Galaxy": "film name", 2014: "year", 'Comedy; ': "genre", "Guardians of the Galaxy 2014.csv": "file_name"})
df_all = df_all.drop(columns=['file_name','username', 'helpful', 'total', 'date','title'])
df_all['genre'] = df_all['genre'].str.replace(';','',regex=True)
df_all.head()

,film name,year,genre,rating,review
0,Guardians of the Galaxy,2014,Comedy,10,Guardians of the Galaxy is nothing short of an...
1,Guardians of the Galaxy,2014,Comedy,9,I originally saw in the theater which was grea...
2,Guardians of the Galaxy,2014,Comedy,9,I remember sitting in Hall H at Comic-Con two ...
3,Guardians of the Galaxy,2014,Comedy,9,Expectations: surpassed - and they were pretty...
4,Guardians of the Galaxy,2014,Comedy,10,I entitle my review A New Hope as I walk out o...


In [18]:
df_all.to_csv('df_all.csv', index=False)

In [19]:
df_all.head()

,film name,year,genre,rating,review
0,Guardians of the Galaxy,2014,Comedy,10,Guardians of the Galaxy is nothing short of an...
1,Guardians of the Galaxy,2014,Comedy,9,I originally saw in the theater which was grea...
2,Guardians of the Galaxy,2014,Comedy,9,I remember sitting in Hall H at Comic-Con two ...
3,Guardians of the Galaxy,2014,Comedy,9,Expectations: surpassed - and they were pretty...
4,Guardians of the Galaxy,2014,Comedy,10,I entitle my review A New Hope as I walk out o...


In [20]:
# Filter for 'Comedy' genre only (excluding rows that also contain 'Horror')
df_romcom = df_all[df_all['genre'].str.contains('Comedy') & ~df_all['genre'].str.contains('Horror')]

# Filter for 'Horror' genre only (excluding rows that also contain 'Comedy')
df_horror = df_all[df_all['genre'].str.contains('Horror') & ~df_all['genre'].str.contains('Comedy')]

In [21]:
df_romcom.head()

,film name,year,genre,rating,review
0,Guardians of the Galaxy,2014,Comedy,10,Guardians of the Galaxy is nothing short of an...
1,Guardians of the Galaxy,2014,Comedy,9,I originally saw in the theater which was grea...
2,Guardians of the Galaxy,2014,Comedy,9,I remember sitting in Hall H at Comic-Con two ...
3,Guardians of the Galaxy,2014,Comedy,9,Expectations: surpassed - and they were pretty...
4,Guardians of the Galaxy,2014,Comedy,10,I entitle my review A New Hope as I walk out o...


In [22]:
df_horror.head()

,film name,year,genre,rating,review
78795,The Shining,1980,Drama Horror,rating,review
78796,The Shining,1980,Drama Horror,Null,"When this film first came out in 1980, I remem..."
78797,The Shining,1980,Drama Horror,Null,I can't praise this film long enough!<br/><br/...
78798,The Shining,1980,Drama Horror,Null,Stephen King may have said the master director...
78799,The Shining,1980,Drama Horror,9,Even though The Shining is over a quarter of a...


We have 174941 rows of movie reviews, with 5 columns of information: 
- Film Name
- Year
- Genre
- Rating
- Review


In [23]:
#TODO: We need to add the header in to the appended reviews file.

In [26]:

# Apply the text_to_sentiment function to each review in the df_romcom DataFrame
df_romcom['sentiment'] = df_romcom['review'].apply(text_to_sentiment)

# Calculate the average sentiment for comedies
average_sentiment_comedy = df_romcom['sentiment'].mean()

# Apply the text_to_sentiment function to each review in the df_horror DataFrame
df_horror['sentiment'] = df_horror['review'].apply(text_to_sentiment)

# Calculate the average sentiment for horror movies
average_sentiment_horror = df_horror['sentiment'].mean()

# Print the average sentiments
print(f"Average Sentiment for Comedies: {average_sentiment_comedy}")
print(f"Average Sentiment for Horror Movies: {average_sentiment_horror}")

In [25]:
#Average Sentiment for Comedies: 0.36794964103811045
average_sentiment_comedy = 0.36794964103811045
#Average Sentiment for Horror Movies: -0.15294894375162382
average_sentiment_horror = -0.15294894375162382

## Analysis

TODO: Add analysis

## Conclusion

TODO: Add conclusion

Identify two conferences or journals that would be interested in the results of your analysis.  

## References